In [3]:
import os

In [4]:
import json
import re
from google.cloud import vision
from google.cloud import storage

In [5]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="""C:/Users/Alexander Lawson/Downloads/calendaralerts-301014-5cbdeed27a17.json"""

In [14]:
gcs_source_uri="""gs://equibase_pdfs_1/test2.pdf"""

In [15]:
# Supported mime_types are: 'application/pdf' and 'image/tiff'
mime_type = 'application/pdf'

# How many pages should be grouped into each json output file.
batch_size = 10

client = vision.ImageAnnotatorClient()

feature = vision.Feature(
    type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

In [16]:
gcs_source = vision.GcsSource(uri=gcs_source_uri)
input_config = vision.InputConfig(
    gcs_source=gcs_source, mime_type=mime_type)

In [17]:
gcs_destination_uri="""gs://equibase_pdfs_1/jsons_out/test2"""

In [18]:
gcs_destination = vision.GcsDestination(uri=gcs_destination_uri)
output_config = vision.OutputConfig(
    gcs_destination=gcs_destination, batch_size=batch_size)

In [19]:
async_request = vision.AsyncAnnotateFileRequest(
    features=[feature], input_config=input_config,
    output_config=output_config)

In [20]:
operation = client.async_batch_annotate_files(requests=[async_request])

In [21]:
operation.result(timeout=420)

responses {
  output_config {
    gcs_destination {
      uri: "gs://equibase_pdfs_1/jsons_out/test2"
    }
    batch_size: 10
  }
}

In [22]:
storage_client = storage.Client()

match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
bucket_name = match.group(1)
prefix = match.group(2)

bucket = storage_client.get_bucket(bucket_name)


In [23]:
blob_list = list(bucket.list_blobs(prefix=prefix))

In [24]:
output = blob_list[0]

In [25]:
json_string = output.download_as_string()
response = json.loads(json_string)

In [26]:
blob=bucket.blob('text_files_out/test2.txt')

In [27]:
blob.upload_from_string(response['responses'][0]['fullTextAnnotation']['text'])

In [28]:
a=(response['responses'][0]['fullTextAnnotation']['text'])

In [33]:
a[a.find('Past Performance Running Line Preview'):a.find('Trainers')]

'Past Performance Running Line Preview\nPgm Horse Name Start 1/4 1/2 314\n5 Colonel Liam\n12\n72 1/4\n6 Largent\n2\n9 Cross Border\n7\n12 Social Paranoia\n6\n10 Pixelate\n5\n1 Next Shares\n10\n3 Storm the Court\n1\n7 Aquaphobia\n8\n2 Breaking the Rules 4\n4 North Dakota\n11\n127\n11 Say the Word\n9\n73 1/4\n8 Anothertwistafate 3\n21\n53\n32\n53\n116\n17\n1\n2 Head\n62 1/4\n63 1/4\n42\n72 3/4\n41\n84 1/4\n31/2\n82 3/4\n1 Head\n41\n31\n72 1/4\n115\n32 1/4\n51 1/2\n1 Head\n42 3/4\n61314\n103 3/4\n83 114\n114 1/2\n11/2\n105\n103 112\n63 1/4\n114 114\n94 3/4\n126 1/2\n73 1/2\n93 1/4\n93 3/4\n125 1/4\n82 3/4\n125 1/2\n94 1/4\n103 1/2\n104 1/4\n115 112\n93 1/4\n2 Head\n21/2\n5314\n126\n'